# Solução do modelo de barganha de Dou et al. (2019)




In [1]:
#para mostrar todos os resultados e não apenas o último
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"


In [3]:
#libraries
import numpy as np
import scipy

import quantecon as qe





In [5]:
#libraries
import numpy as np
from numba import jit, jitclass, float64, njit
import matplotlib.pyplot as plt
%matplotlib inline
import quantecon as qe
from quantecon.distributions import BetaBinomial

In [1]:
print("Olá, mundo")

Olá, mundo


## Algoritmo

1. Sortear o credor que proporá neste turno, com chance $\lambda_j$

2. Fazer o credor sorteado escolher entre Liquidar ou Reorganizar a firma, de acordo com a sua equação de Bellman

4. Verificar se o credor adversário aceita ou rejeita:
    1. Se aceitar, os payoffs são realizados e o jogo acaba
    2. Se rejeitar, passamos para o próximo turno
    






In [ ]:
#parâmetros iniciais

#parâmetros iniciais

n, a, b = 50, 200, 100                        # default parameters
q_default = BetaBinomial(n, a, b).pdf()       # distribuição Beta Binomial


In [ ]:
#vamos descrever o tipo de dados que usaremos para facilitar a vida do Numba

dou_data = [
    ('Ls', float64),     # valor de liquidação para s
    ('Lj', float64),     # valor de liquidação para j
    ('ρ', float64),      #fração do valor da firma que sobrevive a cada período
    ('β', float64),      # parâmetro Beta
    ('v', float64[:]),   # array com os valores da firma, v[t] = valor no estágio t
    ('q', float64[:])    # array of probabilities
]


In [ ]:
#código do modelo de McCall


@jitclass(dou_data)
class DouModel:

    def __init__(self, c=25, β=9, w=w_default, q=q_default):

        self.c, self.β = c, β
        self.w, self.q = w_default, q_default
    def bellman(self, i, v):
        """
        The r.h.s. of the Bellman equation at state i.
        """
        # Simplify names
        c, β, w, q = self.c, self.β, self.w, self.q
        # Evaluate right hand side of Bellman equation
        max_value = max(w[i] / (1 - β), c + β * np.sum(v * q))
        return(max_value)